In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score,average_precision_score
import lightgbm as lgb
import catboost as cb
from sklearn.model_selection import StratifiedKFold
import warnings
warnings.filterwarnings('ignore')

train_x = pd.read_csv('creditcard_train.csv')
test_x = pd.read_csv('creditcard_test.csv')
res = test_x[['Index']]
del train_x['Index']
del test_x['Index']

test_x['Class'] = -1
data = pd.concat([train_x,test_x],axis=0,ignore_index=True)
train_index = data[data['Class']!=-1].index.tolist()
test_index = data[data['Class']==-1].index.tolist()
data['Time'] = data['Time'].astype(int)
data['Day'] = (data['Time']//(3600*24)).astype(int)
data['Hour'] = ((data['Time']-data['Day']*24*3600)//(3600)).astype(int)



def get_pywt_1(x):
    import pywt
    db1 = pywt.Wavelet('db1')
    A= pywt.wavedec(x, db1,level=4)
    return A[0][0]
def get_pywt_2(x):
    import pywt
    db1 = pywt.Wavelet('db1')
    A= pywt.wavedec(x, db1,level=4)
    return A[0][1]
def get_pywt_3(x):
    import pywt
    db1 = pywt.Wavelet('db1')
    A= pywt.wavedec(x, db1,level=4)
    return A[1][0]
def get_pywt_4(x):
    import pywt
    db1 = pywt.Wavelet('db1')
    A= pywt.wavedec(x, db1,level=4)
    return A[1][1]
columns=['V'+str(i) for i in range(1,29)]
data['xiao1']=data[columns].apply(get_pywt_1,axis=1)
data['xiao2']=data[columns].apply(get_pywt_2,axis=1)
data['xiao3']=data[columns].apply(get_pywt_3,axis=1)
data['xiao4']=data[columns].apply(get_pywt_4,axis=1)


pca_cols = []
for i in range(1,29):
    pca_cols.append('V'+str(i))
def getFeature(x):
    return ' '.join(x.astype(str).values.tolist())
se  = data[pca_cols].apply(getFeature,axis=1)
st = pd.Series(se.drop_duplicates().values)

data['Id'] = se.map(pd.Series(st.index,index=st.values))

for co in ['Id','Amount']:
    data[co+'_ount'] = data[co].map(data[co].value_counts())

train_x = data.drop(['Id'],axis=1).loc[train_index].reset_index(drop=True)
test_x = data.drop(['Id'],axis=1).loc[test_index].reset_index(drop=True)
train_y = train_x.pop('Class')




del test_x['Class']
print(train_x.shape)
print(test_x.shape)

(256327, 38)
(28480, 38)


0:	test: 0.9557861	best: 0.9557861 (0)	total: 462ms	remaining: 3m 50s
50:	test: 0.9768442	best: 0.9768442 (50)	total: 17.5s	remaining: 2m 33s
100:	test: 0.9786028	best: 0.9793894 (84)	total: 36.1s	remaining: 2m 22s
150:	test: 0.9772216	best: 0.9793894 (84)	total: 53.8s	remaining: 2m 4s
200:	test: 0.9791262	best: 0.9799234 (185)	total: 1m 11s	remaining: 1m 46s
250:	test: 0.9789227	best: 0.9799234 (185)	total: 1m 30s	remaining: 1m 29s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.9799233899
bestIteration = 185

Shrink model to first 186 iterations.


Exception: Data must be 1-dimensional

In [19]:
def searchBestCut(ytrue,ypre):
    lst = []
    for i in range(10,81,1):
        lst.append(f1_score(ytrue,(pd.Series(ypre)>=i*0.01).astype(int).values))
    se = pd.Series(lst,index=range(10,81,1)).sort_values()
    return (se.index[-1])*0.01,se.values[-1]
prob = pd.DataFrame()
cate = pd.DataFrame()
num = 0
apscore = []
fscore = []

X = np.array(train_x)
Y = np.array(train_y)
X_test = np.array(test_x)

skf = StratifiedKFold(n_splits=5,random_state=2020,shuffle=True)
for train_part_index,evals_index in skf.split(train_x,train_y):
    EVAL_RESULT = {}
    X_train, X_val, y_train, y_val = X[train_part_index], X[evals_index], Y[train_part_index], Y[evals_index]

    model = cb.CatBoostClassifier( iterations=500,
                                   depth = 8,
                                   learning_rate = 0.1,
                                   custom_loss='AUC',
                                   eval_metric='AUC',
                                   bagging_temperature=0.83,
                                   od_type='Iter',
                                   rsm = 0.8,
                                   od_wait= 100,
                                   metric_period = 50,
                                   l2_leaf_reg = 5,
                                   thread_count = 50,
                                   random_seed = 2018
                              )

    model.fit(X_train, y_train, eval_set=(X_val, y_val),use_best_model=True)
    
    xx_pred = model.predict_proba(X_val)[:,1].reshape((X_val.shape[0],1))
    test_ypre = model.predict_proba(X_test)[:,1].reshape((X_test.shape[0],1))
    num+=1

    apscore.append(average_precision_score(y_val,xx_pred,average='weighted'))
    
    cut,best_score = searchBestCut(y_val,xx_pred[:,0])
    fscore.append(best_score)
    cate['cate_'+str(num)] = (pd.Series(test_ypre[:,0])>=cut).astype(int)
    prob['prob_'+str(num)]=test_ypre[:,0]
    

print(apscore)
print(sum(apscore)/5)
print(fscore)
print(sum(fscore)/5)
res['Pred'] = prob.mean(1)
res['Class'] = (cate.mean(1)>=0.6).astype(int)
res.to_csv('result/Test_Pred_Class_Cat.csv',index=False)

0:	test: 0.9557861	best: 0.9557861 (0)	total: 187ms	remaining: 1m 33s
50:	test: 0.9768442	best: 0.9768442 (50)	total: 12.8s	remaining: 1m 52s
100:	test: 0.9786028	best: 0.9793894 (84)	total: 26.7s	remaining: 1m 45s
150:	test: 0.9772216	best: 0.9793894 (84)	total: 40.6s	remaining: 1m 33s
200:	test: 0.9791262	best: 0.9799234 (185)	total: 55.6s	remaining: 1m 22s
250:	test: 0.9789227	best: 0.9799234 (185)	total: 1m 9s	remaining: 1m 8s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.9799233899
bestIteration = 185

Shrink model to first 186 iterations.


0:	test: 0.9264799	best: 0.9264799 (0)	total: 362ms	remaining: 3m
50:	test: 0.9681660	best: 0.9683816 (49)	total: 14.4s	remaining: 2m 6s
100:	test: 0.9808358	best: 0.9812901 (96)	total: 27.2s	remaining: 1m 47s
150:	test: 0.9838321	best: 0.9839219 (149)	total: 41.4s	remaining: 1m 35s
200:	test: 0.9840470	best: 0.9847364 (180)	total: 54.2s	remaining: 1m 20s
250:	test: 0.9820076	best: 0.9847364 (180)	total: 1m 8s	remaining: 1m 7s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.9847363842
bestIteration = 180

Shrink model to first 181 iterations.


0:	test: 0.9323340	best: 0.9323340 (0)	total: 379ms	remaining: 3m 9s
50:	test: 0.9843015	best: 0.9850813 (35)	total: 16.4s	remaining: 2m 24s
100:	test: 0.9818963	best: 0.9850813 (35)	total: 28.5s	remaining: 1m 52s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.9850812986
bestIteration = 35

Shrink model to first 36 iterations.


0:	test: 0.9337383	best: 0.9337383 (0)	total: 130ms	remaining: 1m 4s
50:	test: 0.9649165	best: 0.9653795 (47)	total: 12.7s	remaining: 1m 51s
100:	test: 0.9701626	best: 0.9704513 (99)	total: 25.8s	remaining: 1m 41s
150:	test: 0.9729506	best: 0.9743195 (130)	total: 38.6s	remaining: 1m 29s
200:	test: 0.9748953	best: 0.9758527 (181)	total: 51.2s	remaining: 1m 16s
250:	test: 0.9749863	best: 0.9758527 (181)	total: 1m 3s	remaining: 1m 3s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.9758527446
bestIteration = 181

Shrink model to first 182 iterations.


0:	test: 0.9461038	best: 0.9461038 (0)	total: 146ms	remaining: 1m 12s
50:	test: 0.9818025	best: 0.9818025 (50)	total: 13.6s	remaining: 1m 59s
100:	test: 0.9826217	best: 0.9835146 (65)	total: 26.3s	remaining: 1m 43s
150:	test: 0.9834255	best: 0.9835146 (65)	total: 39.5s	remaining: 1m 31s
200:	test: 0.9835954	best: 0.9839422 (188)	total: 52.1s	remaining: 1m 17s
250:	test: 0.9845990	best: 0.9848060 (242)	total: 1m 5s	remaining: 1m 4s
300:	test: 0.9853705	best: 0.9854200 (297)	total: 1m 16s	remaining: 50.5s
350:	test: 0.9859074	best: 0.9860637 (327)	total: 1m 30s	remaining: 38.3s
400:	test: 0.9859225	best: 0.9865471 (390)	total: 1m 43s	remaining: 25.4s
450:	test: 0.9860717	best: 0.9865471 (390)	total: 1m 57s	remaining: 12.7s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.9865470942
bestIteration = 390

Shrink model to first 391 iterations.
[0.8982797682150906, 0.8250267642221921, 0.830060000314953, 0.8331338115158007, 0.929535631916322]
0.8632071952368717
[0.878612716

IOError: [Errno 2] No such file or directory: 'result/Test_Pred_Class_Cat.csv'

In [21]:
res.to_csv('Test_Pred_Class_Cat.csv',index=False)